In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
data = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/IMDB Dataset.csv')

In [10]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [11]:
data.shape

(50000, 2)

In [12]:
type(data)

pandas.core.frame.DataFrame

In [13]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


#One Hot Encoding
#Label Encoder

In [16]:
data.replace({"sentiment" : {"positive" : 1, "negative" : 0}}, inplace=True)

In [17]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


#Data Preprocessing

In [18]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [19]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [20]:
train_data.shape

(40000, 2)

In [22]:
test_data.shape

(10000, 2)

In [23]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data['review'])

In [29]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen=200)

In [30]:
X_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32)

In [31]:
X_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [  12,  162,   59, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]], dtype=int32)

In [32]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [33]:
Y_train

,sentiment
39087,0
30893,0
45278,1
16398,0
13653,0
...,...
11284,1
44732,1
38158,0
860,1


#Model Building

In [34]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [35]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [36]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [37]:
model.fit(X_train, Y_train, batch_size=64, epochs=5, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 223s 426ms/step - accuracy: 0.7252 - loss: 0.5295 - val_accuracy: 0.8525 - val_loss: 0.3465
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 204s 408ms/step - accuracy: 0.8563 - loss: 0.3505 - val_accuracy: 0.8575 - val_loss: 0.3316
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 203s 407ms/step - accuracy: 0.8838 - loss: 0.2911 - val_accuracy: 0.8639 - val_loss: 0.3454
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 203s 407ms/step - accuracy: 0.9011 - loss: 0.2535 - val_accuracy: 0.8701 - val_loss: 0.3251
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 205s 409ms/step - accuracy: 0.9114 - loss: 0.2305 - val_accuracy: 0.8720 - val_loss: 0.3131


In [38]:
loss, accuracy = model.evaluate(X_test, Y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 45s 141ms/step - accuracy: 0.8735 - loss: 0.2988


In [39]:
print(loss)

0.2982965409755707


In [40]:
print(accuracy)

0.8762000203132629


#Building Predictive System

In [42]:
def predictive_system(review):
  sequences = tokenizer.texts_to_sequences([review])
  padded_sequences = pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequences)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [43]:
predictive_system("This movie was fantastic and amazing")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


'positive'

In [44]:
predictive_system("A trilling adventure with stunning visual")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step


'positive'

In [45]:
predictive_system("A visual masterpiece")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step


'positive'

#Saving Model

In [46]:
model.save("model.h5")

In [47]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']